In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

# 데이터 불러오기
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')

# 농가구역 One-hot encoding
input_data = pd.get_dummies(input_data, columns=['frmDist'], drop_first=False)

# na값 drop
input_data = input_data.dropna()

# 특성과 타겟 변수 설정
X = input_data.drop(columns=['outtrn_cumsum', 'HeatingEnergyUsage_cumsum'])
y_outtrn = input_data['outtrn_cumsum']
y_heating = input_data['HeatingEnergyUsage_cumsum']

# 트레이닝, 테스트 데이터 분리
X_train, X_test, y_train_outtrn, y_test_outtrn, y_train_heating, y_test_heating = train_test_split(
    X, y_outtrn, y_heating, test_size=0.2, random_state=42
)

# 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 다양한 모델 시도
models = [
    ('RandomForest', RandomForestRegressor(random_state=42)),
    ('GradientBoosting', GradientBoostingRegressor(random_state=42))
]

best_model = None
best_rmse = float('inf')
best_r2 = -float('inf')

for model_name, model in models:
    model.fit(X_train_scaled, y_train_outtrn)  # outtrn_cumsum 예측
    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test_outtrn, y_pred))
    r2 = r2_score(y_test_outtrn, y_pred)
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_r2 = r2
        best_model = model_name

print("Best Model for outtrn_cumsum:", best_model)
print("Best RMSE for outtrn_cumsum:", best_rmse)
print("Best R-squared (R2) for outtrn_cumsum:", best_r2)

# 앙상블 모델 생성
voting_regressor = VotingRegressor(estimators=models)

# 하이퍼파라미터 그리드 설정
param_grid = {
    # 'RandomForest__n_estimators': [100, 200],
    'GradientBoosting__n_estimators': [100, 200]
}

# 그리드 서치 객체 생성
grid_search = GridSearchCV(voting_regressor, param_grid, cv=5, n_jobs=-1, verbose=1, scoring='neg_mean_squared_error')

# 그리드 서치 수행
grid_search.fit(X_train_scaled, y_train_outtrn)

# 최적의 앙상블 모델 선택
best_ensemble = grid_search.best_estimator_

# 테스트 데이터로 예측
y_pred = best_ensemble.predict(X_test_scaled)

# 평가
rmse = np.sqrt(mean_squared_error(y_test_outtrn, y_pred))
r2 = r2_score(y_test_outtrn, y_pred)
print("Best Ensemble Model RMSE for outtrn_cumsum:", rmse)
print("Best Ensemble Model R-squared (R2) for outtrn_cumsum:", r2)


Best Model for outtrn_cumsum: RandomForest
Best RMSE for outtrn_cumsum: 3842.4674033849196
Best R-squared (R2) for outtrn_cumsum: 0.9902779416062468
Fitting 5 folds for each of 2 candidates, totalling 10 fits
